In [ ]:
import mlflow
from mlflow.tracking import MlflowClient
import sys
import os

# ==================== CONFIGURATION ====================

# 💡 महत्वपूर्ण: यह Experiment Name, model_train.py में प्रयुक्त नाम से मेल खाना चाहिए।
EXPERIMENT_NAME = "/Shared/House_Price_Prediction_Delta_RF"

# ⚠️ UNITY CATALOG के लिए नामकरण (MANDATORY)
UC_CATALOG_NAME = "workspace"
UC_SCHEMA_NAME = "ml"
REGISTERED_MODEL_NAME = f"{UC_CATALOG_NAME}.{UC_SCHEMA_NAME}.house_price_model_uc" 

MODEL_ARTIFACT_PATH = "sklearn_rf_model"

# मॉडल पैरामीटर जिन्हें हम तुलना के लिए उपयोग करेंगे (model_train.py से)
PARAM_KEYS = ['n_estimators', 'max_depth']

# ==================== FUNCTIONS ====================

def normalize_param_value(value):
    """
    पैरामीटर की value को normalize करता है ताकि comparison सही हो सके।
    MLflow कभी-कभी parameters को string के रूप में store करता है।
    """
    if value is None:
        return None
    
    # String को appropriate type में convert करें
    str_value = str(value)
    
    # Check if it's a number
    try:
        # पहले integer try करें
        if '.' not in str_value:
            return int(str_value)
        else:
            return float(str_value)
    except (ValueError, TypeError):
        # यदि number नहीं है तो string ही return करें
        return str_value


def get_latest_run_id(client: MlflowClient):
    """
    निर्दिष्ट एक्सपेरिमेंट में हाल ही में सफल हुए रन का ID प्राप्त करता है।
    """
    try:
        experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
        if not experiment:
            print(f"❌ Error: Experiment '{EXPERIMENT_NAME}' not found.")
            return None

        # सभी रन को शुरू होने के समय के अनुसार क्रमबद्ध करें
        runs = client.search_runs(
            experiment_ids=[experiment.experiment_id],
            order_by=["start_time DESC"],
            max_results=1
        )
        
        if runs:
            latest_run = runs[0]
            print(f"✓ Latest Run ID found: {latest_run.info.run_id}")
            return latest_run.info.run_id
        else:
            print(f"⚠ Warning: No runs found in experiment '{EXPERIMENT_NAME}'.")
            return None
            
    except Exception as e:
        print(f"❌ Error fetching latest run ID: {e}")
        return None


def get_run_params_from_version(client: MlflowClient, model_name: str, version_number: str):
    """
    Model version से associated run के parameters निकालता है।
    """
    try:
        version_details = client.get_model_version(model_name, version_number)
        
        # Version से run_id निकालें
        run_id = version_details.run_id
        
        if not run_id:
            print(f"⚠ Warning: No run_id found for version {version_number}")
            return {}
        
        # Run से parameters निकालें
        run = client.get_run(run_id)
        run_params = {}
        
        for key in PARAM_KEYS:
            if key in run.data.params:
                param_value = run.data.params[key]
                run_params[key] = normalize_param_value(param_value)
        
        return run_params
        
    except Exception as e:
        print(f"⚠ Warning: Could not fetch params for version {version_number}: {e}")
        return {}


def check_existing_version(client: MlflowClient, run_params: dict):
    """
    जाँच करता है कि क्या समान पैरामीटर वाला मॉडल संस्करण पहले से ही पंजीकृत है।
    UC में, हम सभी संस्करणों को मैन्युअल रूप से फ़ेच करके जाँच करते हैं।
    """
    try:
        # Current run के parameters को normalize करें
        normalized_run_params = {
            k: normalize_param_value(v) for k, v in run_params.items()
        }
        
        print(f"\n🔍 Checking for existing versions with parameters: {normalized_run_params}")
        
        # Unity Catalog में सभी model versions खोजें
        filter_string = f"name = '{REGISTERED_MODEL_NAME}'"
        versions = client.search_model_versions(filter_string=filter_string)
        
        if not versions:
            print(f"ℹ️ Model '{REGISTERED_MODEL_NAME}' does not exist in Unity Catalog. Proceeding with registration.")
            return None
        
        print(f"📋 Found {len(versions)} existing version(s). Checking parameters...")
        
        for version in versions:
            try:
                # Version से associated run के parameters निकालें
                version_params = get_run_params_from_version(
                    client, 
                    REGISTERED_MODEL_NAME, 
                    version.version
                )
                
                if not version_params:
                    print(f"   Version {version.version}: No parameters found, skipping...")
                    continue
                
                print(f"   Version {version.version}: Parameters = {version_params}")
                
                # Parameters की तुलना करें
                params_match = True
                for key in PARAM_KEYS:
                    if key not in version_params or key not in normalized_run_params:
                        params_match = False
                        break
                    if version_params[key] != normalized_run_params[key]:
                        params_match = False
                        break
                
                if params_match:
                    print("\n" + "#" * 70)
                    print(f"⏭️  DUPLICATE DETECTED: Version {version.version} already exists")
                    print(f"   with identical parameters in Unity Catalog.")
                    print(f"   Existing: {version_params}")
                    print(f"   Current:  {normalized_run_params}")
                    print(f"   ✅ Skipping registration to avoid duplicate.")
                    print("#" * 70 + "\n")
                    return version
                    
            except Exception as e:
                print(f"   ⚠ Error checking version {version.version}: {e}")
                continue
                
        print("✓ No duplicate found. Proceeding with new registration.\n")
        return None  # कोई डुप्लीकेट नहीं मिला
        
    except Exception as e:
        # UC में यह अक्सर 'RESOURCE_DOES_NOT_EXIST' या अनुमति त्रुटि (Permission error) देगा
        error_msg = str(e).upper()
        if "RESOURCE_DOES_NOT_EXIST" in error_msg or "PERMISSION_DENIED" in error_msg:
            print(f"ℹ️ Model '{REGISTERED_MODEL_NAME}' not found or Permission Denied in UC.")
            print(f"   Proceeding with registration as first version.\n")
            return None
        
        # अन्य errors के लिए
        print(f"⚠ Warning: Error checking existing versions in UC: {e}")
        print(f"   Proceeding with registration (caution advised).\n")
        return None


def register_model_for_serving(client: MlflowClient, run_id: str, model_name: str, artifact_path: str):
    """
    MLflow Run से मॉडल को Unity Catalog Model Registry में पंजीकृत करता है।
    """
    
    # रन के पैरामीटर प्राप्त करें
    run = client.get_run(run_id)
    run_params = {k: run.data.params.get(k) for k in PARAM_KEYS}
    print(f"\n⚙️  Latest Run Parameters: {run_params}")
    
    # 1. डुप्लीकेट जाँच
    existing_version = check_existing_version(client, run_params)
    if existing_version:
        print(f"🎯 Using existing model version: {existing_version.name} v{existing_version.version}")
        return existing_version

    # 2. पंजीकरण
    model_uri = f"runs:/{run_id}/{artifact_path}"
    print(f"⏳ Attempting to register NEW model version from URI: {model_uri}")
    
    try:
        # MLflow स्वतः ही UC का उपयोग करेगा
        model_version = mlflow.register_model(
            model_uri=model_uri, 
            name=model_name
        )
        
        print("\n" + "=" * 60)
        print("✅ UC MODEL REGISTRATION SUCCESSFUL!")
        print(f"नाम: {model_version.name}")
        print(f"संस्करण (Version): {model_version.version}")
        print(f"स्टेज (Stage): {model_version.current_stage}")
        print(f"Parameters: {run_params}")
        print("=" * 60 + "\n")
        
        return model_version
        
    except Exception as e:
        print(f"❌ UC मॉडल पंजीकरण विफल हुआ! त्रुटि: {e}")
        print("सुनिश्चित करें कि आपके पास Unity Catalog में मॉडल बनाने की अनुमति है।")
        sys.exit(1)


if __name__ == "__main__":
    
    print("\n" + "=" * 60)
    print("🚀 MLFLOW MODEL REGISTRATION WITH DUPLICATE CHECK")
    print("=" * 60 + "\n")
    
    # UC के लिए पुराने Workspace Registry कॉन्फ़िगरेशन को हटा दें
    try:
        if "MLFLOW_REGISTRY_URI" in os.environ:
            del os.environ["MLFLOW_REGISTRY_URI"]
            print("✓ Removed MLFLOW_REGISTRY_URI environment variable for UC registration.")
    except Exception as e:
        print(f"⚠ Warning: Could not remove MLFLOW_REGISTRY_URI: {e}")

    # Mlflow क्लाइंट को इनिशियलाइज़ करें
    client = MlflowClient()

    # 1. रन ID को स्वचालित रूप से प्राप्त करें
    RUN_ID = get_latest_run_id(client)
    
    if RUN_ID:
        # डुप्लीकेट चेक और पंजीकरण (Registration)
        registered_version = register_model_for_serving(
            client, 
            RUN_ID, 
            REGISTERED_MODEL_NAME, 
            MODEL_ARTIFACT_PATH
        )
        
        if registered_version:
            print(f"\n✅ Final Result:")
            print(f"   Model: {registered_version.name}")
            print(f"   Version: {registered_version.version}")
            print(f"\n💡 अगला कदम: इस UC मॉडल संस्करण का उपयोग करके Serving Endpoint बनाएँ।")
    else:
        print("❌ Registration aborted: Could not retrieve a valid Run ID.")